In [ ]:
# This cell is mandatory in all Dymos documentation notebooks.
missing_packages = []
try:
    import openmdao.api as om  # noqa: F401
except ImportError:
    if 'google.colab' in str(get_ipython()):
        !python -m pip install openmdao[notebooks]
    else:
        missing_packages.append('openmdao')
try:
    import dymos as dm  # noqa: F401
except ImportError:
    if 'google.colab' in str(get_ipython()):
        !python -m pip install dymos
    else:
        missing_packages.append('dymos')
try:
    import pyoptsparse  # noqa: F401
except ImportError:
    if 'google.colab' in str(get_ipython()):
        !pip install -q condacolab
        import condacolab
        condacolab.install_miniconda()
        !conda install -c conda-forge pyoptsparse
    else:
        missing_packages.append('pyoptsparse')
if missing_packages:
    raise EnvironmentError('This notebook requires the following packages '
                           'please install them and restart this notebook\'s runtime: {",".join(missing_packages)}')

## Simulating a cannonball using PicardShooting.

Below we implement cartesian, ballistic equations of motion using OpenMDAO's `JaxExplicitComponent` so that we can use automatic differentiation to provide the partials.

In [ ]:
import jax.numpy as jnp
import openmdao.api as om


class BallisticEOM(om.JaxExplicitComponent):
    def initialize(self):
        self.options.declare('num_nodes', types=int)

    def setup(self):
        nn = self.options['num_nodes']

        self.add_input('y', shape=(nn,), units='m')
        self.add_input('vx', shape=(nn,), units='m/s')
        self.add_input('vy', shape=(nn,), units='m/s')
        self.add_input('c_d', units='unitless')
        self.add_input('A', val=0.001, units='m**2')
        self.add_input('m', val=1, units='kg')
        self.add_input('g', val=9.80665, units='m/s**2')
        self.add_input('rho_0', val=1.22, units='kg/m**3')

        self.add_output('x_dot', shape=(nn,), units='m/s', tags=['dymos.state_rate_source:x', 'dymos.state_units:m'])
        self.add_output('y_dot', shape=(nn,), units='m/s', tags=['dymos.state_rate_source:y', 'dymos.state_units:m'])
        self.add_output('vx_dot', shape=(nn,), units='m/s**2', tags=['dymos.state_rate_source:vx', 'dymos.state_units:m/s'])
        self.add_output('vy_dot', shape=(nn,), units='m/s**2', tags=['dymos.state_rate_source:vy', 'dymos.state_units:m/s'])
        self.add_output('v', shape=(nn,), units='m/s')


    def compute_primal(self, y, vx, vy, c_d, A, m, g, rho_0):
        gamma = jnp.arctan2(vy, vx)
        rho = rho_0*jnp.exp(-y/8500)
        v = jnp.sqrt(vx**2 + vy**2)
        D = 0.5*rho*(v**2)*c_d*A

        vx_dot = (-D/m) * jnp.cos(gamma)
        vy_dot = -g - (D/m) * jnp.sin(gamma)
        x_dot = vx
        y_dot = vy

        return x_dot, y_dot, vx_dot, vy_dot, v

## Propagating the equations of motion using Picard iteration.

With the pseudospetral methods, we provide the values of a state variable throughout the trajectory, and can treat the "defect" constraints as residuals to be solved by a Newton solver using the `solve_segments` option in Dymos.

The Newton solver han have a difficult time simultaneously satisfying these defect residuals and residuals associated with "parameters" like the phase duration or phase ODE parameters.

Picard iteration provides the solution to the integral of an ODE through fixed-point iteration:

\begin{align}
   x_{k + 1} &= x_{k} + \int_{t_0}^{t_f} f_{ode}(x_{k}, t, u, p) dt
\end{align}

Leveraging the Birkhoff integration matrix $[B]$, we can treat this as a discretized problem:

\begin{align}
   \bar{x}_{k + 1} &= \bar{x}_{k} + [B] f_{ode}(\bar{x}_{k}, \bar{t}, \bar{u}, p)
\end{align}

This allows us to apply either the Newton solver or the Nonlinear Block Gauss Seidel (NLBGS) in OpenMDAO to compute the integral.
NLBGS can be advantageous because it doesn't require the derivatives to be assembled in the same way the Newton solver does, so it's a bit faster.

### Robustness

PicardIteration works well for many ODEs, but it can struggle when the solution is chaotic.
This integration method is still implicit, so a reasonable initial guess is beneficial.

### Single vs Multiple Shooting

As implemented in Dymos, the forulae above provide the integral across a single segment.
An outer loop around this integration updates the initial state value in each segment to enforce state continuity across segment bounds.
A single instance of the ODE is used to converge the state history via multiple shooting when more than one segment is used.

### Differences between PicardShooting and other transcriptions.

PicardShooting uses a default value of `solve_segments='forward'`. Unless otherwise specified, states will have their `solve_segments` direction set to this value by default.
Only values of `forward` or `backward` are acceptable in PicardShooting.

In [ ]:
# Define the OpenMDAO problem
p = om.Problem(model=om.Group())

# Define a Trajectory object
traj = dm.Trajectory()
p.model.add_subsystem('traj', subsys=traj)

# Define a Dymos Phase object with Radau Transcription
phase = dm.Phase(ode_class=BallisticEOM,
                 transcription=dm.PicardShooting(num_segments=1, nodes_per_seg=21))
traj.add_phase(name='phase0', phase=phase)

# Set the time options
phase.set_time_options(fix_initial=True,
                       duration_bounds=(0.5, 10.0))

# Set the state options
phase.add_state('x', rate_source='x_dot', fix_initial=True)
phase.add_state('y', rate_source='y_dot', fix_initial=True)
phase.add_state('vx', rate_source='vx_dot', fix_initial=False)
phase.add_state('vy', rate_source='vy_dot', fix_initial=False)

phase.add_parameter('m', opt=False, val=1.0, units='kg')
phase.add_parameter('g', opt=False, val=9.80665, units='m/s**2')
phase.add_parameter('c_d', opt=False, val=0.5, units='unitless')
phase.add_parameter('A', opt=False, val=0.001, units='m**2')
phase.add_parameter('rho_0', opt=False, val=1.22, units='kg/m**3')

# Minimize final range.
# phase.add_objective('x', loc='final', scaler=-1)

# Setup the problem
p.setup()

phase.set_time_val(initial=0.0, duration=20.0)
phase.set_state_val('x', vals=[0, 100])
phase.set_state_val('y', vals=[0, 0])
phase.set_state_val('vx', vals=[100, 100])
phase.set_state_val('vy', vals=[100, 0])

# Run the driver to solve the problem
p.run_model()

If we plot the resulting trajectory of the bead, we notice that our guesses for time and the control history didn't bring the bead to the desired target at (10, 5):

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

x = p.get_val('traj.phase0.timeseries.x')
y = p.get_val('traj.phase0.timeseries.y')

plt.plot(0.0, 10.0, 'ko')
plt.plot(10.0, 5.0, 'ko')
plt.plot(x, y)
plt.show()

## Stopping the propagation at the desired time.

We can utilize the `set_implicit_duration` method on `Phase` to turn `t_duration` into an implicit output and provide it with a residual.

In our case, we set `phase.set_duration_balance('y', 0.0)`

This specifies that we want `t_duration` to be associated with the following residual:

\begin{align}
  \mathcal{R}(t_d) = y = 0
\end{align}

Note that we should provide a guess in `t_duration` that is closer to impact than it is to the initial launch of the cannonball.
Otherwise the Newton solver would likely try to push the duration to zero, since that would also satisfy the residual.

In [ ]:
# Define the OpenMDAO problem
p = om.Problem(model=om.Group())

# Define a Trajectory object
traj = dm.Trajectory()
p.model.add_subsystem('traj', subsys=traj)

# Define a Dymos Phase object with Radau Transcription
phase = dm.Phase(ode_class=BallisticEOM,
                 transcription=dm.PicardShooting(num_segments=1, nodes_per_seg=21))
traj.add_phase(name='phase0', phase=phase)

# Set the time options
phase.set_time_options(fix_initial=True,
                       duration_bounds=(0.5, 10.0))

# Set the state options
phase.add_state('x', rate_source='x_dot', fix_initial=True)
phase.add_state('y', rate_source='y_dot', fix_initial=True)
phase.add_state('vx', rate_source='vx_dot', fix_initial=False)
phase.add_state('vy', rate_source='vy_dot', fix_initial=False)

phase.add_parameter('m', opt=False, val=1.0, units='kg')
phase.add_parameter('g', opt=False, val=9.80665, units='m/s**2')
phase.add_parameter('c_d', opt=False, val=0.5, units='unitless')
phase.add_parameter('A', opt=False, val=0.001, units='m**2')
phase.add_parameter('rho_0', opt=False, val=1.22, units='kg/m**3')

# Minimize final range.
# phase.add_objective('x', loc='final', scaler=-1)

phase.set_duration_balance('y', 0.0)

# Setup the problem
p.setup()

phase.set_time_val(initial=0.0, duration=20.0)
phase.set_state_val('x', vals=[0, 100])
phase.set_state_val('y', vals=[0, 0])
phase.set_state_val('vx', vals=[100, 100])
phase.set_state_val('vy', vals=[100, 0])

# Run the driver to solve the problem
p.run_model()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

x = p.get_val('traj.phase0.timeseries.x')
y = p.get_val('traj.phase0.timeseries.y')

plt.plot(0.0, 10.0, 'ko')
plt.plot(10.0, 5.0, 'ko')
plt.plot(x, y)
plt.show()